Import Libraries and Define Constants

In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from torchsummary import summary
from sklearn.model_selection import train_test_split
from PIL import Image
import matplotlib.pyplot as plt

In [2]:
device = torch.device("cpu")
IMAGE_SIZE = 64
BATCH_SIZE = 32
NUM_CLASSES = 16  # Assuming you have 15 different cat breeds
NUM_EPOCHS = 2

Establish path to Directories

In [3]:
data_dir = 'CatBreeds\\Gano-Cat-Breeds-V1_1'
if not os.path.exists(data_dir):
    raise FileNotFoundError(f"The directory '{data_dir}' does not exist.")

Define Dataset

In [4]:
class CatDataset(Dataset):
    def __init__(self, filenames, labels, transform=None):
        self.filenames = filenames
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.filenames)

    def __getitem__(self, idx):
        image = Image.open(self.filenames[idx]).convert('RGB')
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, label

Augment Data

In [5]:
train_transform = transforms.Compose([
    transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
    transforms.RandomRotation(20),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Assuming RGB images
])

# Data preprocessing transformation
preprocess_transform = transforms.Compose([
    transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Assuming RGB images
])

In [6]:
# Initialize lists to store file paths and labels
all_filenames = []
all_labels = []

# Loop through each subdirectory (each breed)
for label in os.listdir(data_dir):
    label_dir = os.path.join(data_dir, label)
    # Loop through each image file in the subdirectory
    for filename in os.listdir(label_dir):
        all_filenames.append(os.path.join(label_dir, filename))
        all_labels.append(label)  # Append the label name (categorical) without conversion

# Convert labels to numerical indices
label_to_index = {label: idx for idx, label in enumerate(set(all_labels))}

# Convert all labels to numerical indices using the dictionary
all_labels_numeric = [label_to_index[label] for label in all_labels]

Create Training and Validation Sets

In [7]:
# Split indices
train_indices, val_indices = train_test_split(np.arange(len(all_filenames)), test_size=0.2, random_state=42)

# Create datasets
train_dataset = CatDataset(np.array(all_filenames)[train_indices], np.array(all_labels_numeric)[train_indices], transform=train_transform)
val_dataset = CatDataset(np.array(all_filenames)[val_indices], np.array(all_labels_numeric)[val_indices], transform=preprocess_transform)

# Create data loaders with samplers
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

print("Train Dataset size:", len(train_dataset))
print("Validation Dataset size:", len(val_indices))

Train Dataset size: 4500
Validation Dataset size: 1125


Define CNN, Loss Function, and Optimizer

In [15]:
device = torch.device("cpu")
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.dropout = nn.Dropout(p=0.3)
        self.fc1 = nn.Linear(128 * 8 * 8, 128)
        self.fc2 = nn.Linear(128, NUM_CLASSES)
    def forward(self, x):
        x = self.pool(nn.functional.relu(self.conv1(x.to(device))))
        #x = self.pool(nn.functional.relu(self.conv1(x)))
        x = self.dropout(x)
        x = self.pool(nn.functional.relu(self.conv2(x)))
        x = self.dropout(x)
        x = self.pool(nn.functional.relu(self.conv3(x)))
        x = self.dropout(x)
        x = x.view(-1, 128 * 8 * 8)
        x = nn.functional.relu(self.fc1(x))
        x = self.fc2(x)
        return x

model = CNN().to(device)
summary(model, (3, 64, 64))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 64, 64]             896
         MaxPool2d-2           [-1, 32, 32, 32]               0
           Dropout-3           [-1, 32, 32, 32]               0
            Conv2d-4           [-1, 64, 32, 32]          18,496
         MaxPool2d-5           [-1, 64, 16, 16]               0
           Dropout-6           [-1, 64, 16, 16]               0
            Conv2d-7          [-1, 128, 16, 16]          73,856
         MaxPool2d-8            [-1, 128, 8, 8]               0
           Dropout-9            [-1, 128, 8, 8]               0
           Linear-10                  [-1, 128]       1,048,704
           Linear-11                   [-1, 16]           2,064
Total params: 1,144,016
Trainable params: 1,144,016
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.05
Forw

In [16]:
# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

Begin Training and Record Metrics

In [17]:
# Define function to calculate accuracy
def calculate_accuracy(outputs, labels):
    _, predicted = torch.max(outputs, 1)
    correct = (predicted == labels).sum().item()
    total = labels.size(0)
    accuracy = correct / total
    return accuracy

# Define function to calculate precision
def calculate_precision(outputs, labels):
    _, predicted = torch.max(outputs, 1)
    true_positives = ((predicted == 1) & (labels == 1)).sum().item()
    false_positives = ((predicted == 1) & (labels == 0)).sum().item()
    precision = true_positives / (true_positives + false_positives)
    return precision

# Define function to calculate recall
def calculate_recall(outputs, labels):
    _, predicted = torch.max(outputs, 1)
    true_positives = ((predicted == 1) & (labels == 1)).sum().item()
    false_negatives = ((predicted == 0) & (labels == 1)).sum().item()
    recall = true_positives / (true_positives + false_negatives)
    return recall

# Define function to calculate false positives and false negatives
def calculate_false_predictions(outputs, labels):
    _, predicted = torch.max(outputs, 1)
    false_positives = ((predicted == 1) & (labels == 0)).sum().item()
    false_negatives = ((predicted == 0) & (labels == 1)).sum().item()
    return false_positives, false_negatives

In [18]:
# Initialize Arrays to keep track of metrics
train_losses = []
val_losses = []
train_accuracies = []
val_accuracies = []
train_false_positives = []
train_false_negatives = []
val_false_positives = []
val_false_negatives = []
train_precisions = []
val_precisions = []
train_recalls = []
val_recalls = []

# Training loop
for epoch in range(NUM_EPOCHS):
    model.train()
    running_loss = 0.0
    total = 0
    correct = 0
    train_false_positives_epoch = 0
    train_false_negatives_epoch = 0
    true_positives_epoch = 0
    false_positives_epoch = 0
    false_negatives_epoch = 0

    for i, (inputs, labels) in enumerate(train_loader, 0):
        optimizer.zero_grad()
        outputs = model(inputs)
        # Convert labels to torch.long
        labels = labels.long()
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        total += labels.size(0)
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == labels).sum().item()
        false_positives, false_negatives = calculate_false_predictions(outputs, labels)
        train_false_positives_epoch += false_positives
        train_false_negatives_epoch += false_negatives
        true_positives_epoch += ((predicted == 1) & (labels == 1)).sum().item()
        false_positives_epoch += ((predicted == 1) & (labels == 0)).sum().item()
        false_negatives_epoch += ((predicted == 0) & (labels == 1)).sum().item()

    train_loss = running_loss / len(train_loader)
    train_accuracy = correct / total
    train_precision = true_positives_epoch / (true_positives_epoch + false_positives_epoch + 1e-10)  # to avoid division by zero
    train_recall = true_positives_epoch / (true_positives_epoch + false_negatives_epoch + 1e-10)  # to avoid division by zero

    # Append false positives and false negatives
    train_false_positives.append(train_false_positives_epoch)
    train_false_negatives.append(train_false_negatives_epoch)

    # Validation
    model.eval()
    val_running_loss = 0.0
    val_total = 0
    val_correct = 0
    val_false_positives_epoch = 0
    val_false_negatives_epoch = 0
    val_true_positives_epoch = 0
    val_false_positives_epoch = 0
    val_false_negatives_epoch = 0

    with torch.no_grad():
        for inputs, labels in val_loader:
            outputs = model(inputs)
            # Convert labels to torch.long
            labels = labels.long()
            val_loss = criterion(outputs, labels)
            val_running_loss += val_loss.item()
            val_total += labels.size(0)
            _, predicted = torch.max(outputs, 1)
            val_correct += (predicted == labels).sum().item()
            false_positives, false_negatives = calculate_false_predictions(outputs, labels)
            val_false_positives_epoch += false_positives
            val_false_negatives_epoch += false_negatives
            val_true_positives_epoch += ((predicted == 1) & (labels == 1)).sum().item()
            val_false_positives_epoch += ((predicted == 1) & (labels == 0)).sum().item()
            val_false_negatives_epoch += ((predicted == 0) & (labels == 1)).sum().item()

    val_loss = val_running_loss / len(val_loader)
    val_accuracy = val_correct / val_total
    val_precision = val_true_positives_epoch / (val_true_positives_epoch + val_false_positives_epoch + 1e-10)  # to avoid division by zero
    val_recall = val_true_positives_epoch / (val_true_positives_epoch + val_false_negatives_epoch + 1e-10)  # to avoid division by zero

    # Append false positives and false negatives
    val_false_positives.append(val_false_positives_epoch)
    val_false_negatives.append(val_false_negatives_epoch)

    # Print and store metrics
    print(f'Epoch {epoch + 1}/{NUM_EPOCHS}: '
          f'Train Loss: {train_loss:.4f}, Train Acc: {train_accuracy:.4f}, '
          f'Val Loss: {val_loss:.4f}, Val Acc: {val_accuracy:.4f}, '
          f'Train Precision: {train_precision:.4f}, Train Recall: {train_recall:.4f}, '
          f'Val Precision: {val_precision:.4f}, Val Recall: {val_recall:.4f}')

    train_losses.append(train_loss)
    val_losses.append(val_loss)
    train_accuracies.append(train_accuracy)
    val_accuracies.append(val_accuracy)
    train_precisions.append(train_precision)
    train_recalls.append(train_recall)
    val_precisions.append(val_precision)
    val_recalls.append(val_recall)

print("Finished training")

# Save the trained model
torch.save(model.state_dict(), 'catModels.pth')


Epoch 1/100: Train Loss: 2.6046, Train Acc: 0.1182, Val Loss: 2.4784, Val Acc: 0.1511, Train Precision: 0.5238, Train Recall: 0.4342, Val Precision: 0.4000, Val Recall: 0.5455
Epoch 2/100: Train Loss: 2.4212, Train Acc: 0.1787, Val Loss: 2.3019, Val Acc: 0.2249, Train Precision: 0.7115, Train Recall: 0.4805, Val Precision: 0.7500, Val Recall: 0.7500
Epoch 3/100: Train Loss: 2.2643, Train Acc: 0.2484, Val Loss: 2.2649, Val Acc: 0.2604, Train Precision: 0.8095, Train Recall: 0.6538, Val Precision: 1.0000, Val Recall: 0.0400


KeyboardInterrupt: 

Plot Metrics

In [ ]:
# Plot training and validation loss
plt.figure(figsize=(10, 5))
plt.plot(range(1, NUM_EPOCHS + 1), train_losses, label='Train Loss')
plt.plot(range(1, NUM_EPOCHS + 1), val_losses, label='Val Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()
plt.show()

In [ ]:
# Plot training and validation accuracy
plt.figure(figsize=(10, 5))
plt.plot(range(1, NUM_EPOCHS + 1), train_accuracies, label='Train Accuracy')
plt.plot(range(1, NUM_EPOCHS + 1), val_accuracies, label='Val Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Training and Validation Accuracy')
plt.legend()
plt.show()

In [ ]:
# Plot training and validation precision
plt.figure(figsize=(10, 5))
plt.plot(range(1, NUM_EPOCHS + 1), train_precisions, label='Train Precision')
plt.plot(range(1, NUM_EPOCHS + 1), val_precisions, label='Val Precision')
plt.xlabel('Epoch')
plt.ylabel('Precision')
plt.title('Training and Validation Precision')
plt.legend()
plt.show()

In [ ]:
# Plot false positives and false negatives
plt.figure(figsize=(10, 5))
plt.plot(range(1, NUM_EPOCHS + 1), train_false_positives, label='Train False Positives')
plt.plot(range(1, NUM_EPOCHS + 1), train_false_negatives, label='Train False Negatives')
plt.plot(range(1, NUM_EPOCHS + 1), val_false_positives, label='Val False Positives')
plt.plot(range(1, NUM_EPOCHS + 1), val_false_negatives, label='Val False Negatives')
plt.xlabel('Epoch')
plt.ylabel('Count')
plt.title('False Positives and False Negatives')
plt.legend()
plt.show()

In [ ]:
# Plot training and validation recall
plt.figure(figsize=(10, 5))
plt.plot(range(1, NUM_EPOCHS + 1), train_recalls, label='Train Recall')
plt.plot(range(1, NUM_EPOCHS + 1), val_recalls, label='Val Recall')
plt.xlabel('Epoch')
plt.ylabel('Recall')
plt.title('Training and Validation Recall')
plt.legend()
plt.show()

Load Model for Testing

In [ ]:
model = CNN()
model.load_state_dict(torch.load('catModels.pth'))
model.eval()

In [ ]:
def predict_breed(image_path):
    # Load and preprocess the test image
    img = Image.open(image_path).convert('RGB')

    # Preprocess the image for the model
    img_tensor = train_transform(img).unsqueeze(0)  # Add batch dimension
    
    # Forward pass through the model
    output = model(img_tensor)
    probabilities = torch.softmax(output, dim=1)
    confidence, predicted_class = torch.max(probabilities, 1)
    
    # Map predicted index to breed label
    index_to_label = {v: k for k, v in label_to_index.items()}
    predicted_breed = index_to_label[predicted_class.item()]
    
    return predicted_breed, confidence.item()


In [ ]:
# Test image path
test_image_path = 'test.jpg'

# Predict breed of the test image with confidence level
prediction, confidence = predict_breed(test_image_path)
print(f"Predicted breed: {prediction}, Confidence: {confidence * 100}%")